In [1]:
%matplotlib inline
import pandas as pd
import os
import networkx as nx
import igraph as ig
import numpy as np
import louvain
import matplotlib.pyplot as plt

In [2]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'
links_df = pd.read_csv(os.path.join(filepath, 'case-to-article-links-unique.csv'))

In [3]:
art_nodes = pd.read_csv(os.path.join(filepath, 'article_nodes_nodup.csv'))
case_nodes = pd.read_csv(os.path.join(filepath, 'case_nodes_simple.csv'))

In [4]:
case_nodes.shape

(502692, 5)

In [5]:
print(art_nodes.shape)
print(art_nodes.title.drop_duplicates().shape)
print(art_nodes.drop('id', axis=1).drop_duplicates().shape)

(597946, 4)
(437867,)
(438609, 3)


In [6]:
print(links_df.shape, links_df.drop_duplicates().shape)

(1750407, 2) (1750407, 2)


In [7]:
# Check that title and label are always equal
art_nodes[art_nodes['title']!=art_nodes['label']]

,id,title,label,authority


In [8]:
# Wich have different authorities?
nr_authorities = art_nodes.groupby(['title', 'label']).nunique()['authority']
nr_authorities[nr_authorities>1].head()

title                                             label                                           
Algemene contributieverordening 2013, Artikel 1   Algemene contributieverordening 2013, Artikel 1     2
Algemene contributieverordening 2013, Artikel 10  Algemene contributieverordening 2013, Artikel 10    2
Algemene contributieverordening 2013, Artikel 11  Algemene contributieverordening 2013, Artikel 11    2
Algemene contributieverordening 2013, Artikel 12  Algemene contributieverordening 2013, Artikel 12    2
Algemene contributieverordening 2013, Artikel 13  Algemene contributieverordening 2013, Artikel 13    2
Name: authority, dtype: int64

In [9]:
art_nodes[art_nodes['title']=='Verordening op het bestuur, Artikel 1']

,id,title,label,authority
113024,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,"Verordening op het bestuur, Artikel 1","Verordening op het bestuur, Artikel 1",Nederlandse Orde van Accountants-Administratie...
340551,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,"Verordening op het bestuur, Artikel 1","Verordening op het bestuur, Artikel 1",Nederlandse Orde van Accountants-Administratie...
466175,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,"Verordening op het bestuur, Artikel 1","Verordening op het bestuur, Artikel 1",Nederlandse beroepsorganisatie van accountants


In [10]:
links_merged = links_df.merge(art_nodes, how='left', left_on='target', right_on='id')
links_merged.head()

,source,target,id,title,label,authority
0,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,Besluit afschaffing binnenlandse paspoorten en...,Besluit afschaffing binnenlandse paspoorten en...,Veiligheid en Justitie
1,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,"Overgangswet, Artikel 1","Overgangswet, Artikel 1",Veiligheid en Justitie
2,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,"Overgangswet, Artikel 1","Overgangswet, Artikel 1",Veiligheid en Justitie
3,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Veiligheid en Justitie
4,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,http://linkeddata.overheid.nl/terms/bwb/id/BWB...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Veiligheid en Justitie


In [11]:
# Group on article name
links_titles = links_merged.groupby(['source', 'title']).count()['id']

In [12]:
# Are there ever citations to multiple versions from one source?
links_titles[links_titles>1].head()

source                                                                         title                                        
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CBB:2001:AB1986  Besluit verdachte dieren, Artikel 2              2
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CBB:2001:AD7632  Besluit biotechnologie bij dieren, Artikel 12    2
                                                                               Besluit biotechnologie bij dieren, Artikel 6     2
http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CBB:2002:AD9458  Meststoffenwet, Artikel 1                        2
                                                                               Meststoffenwet, Artikel 41                       2
Name: id, dtype: int64

In [13]:
# Apparently this happens, look into one
list(links_df[links_df['source']=='http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CBB:2001:AB1986']['target'])

['http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005537/2814224/1994-01-01/1994-01-01',
 'http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005662/2043254/2002-07-05/2002-07-05',
 'http://linkeddata.overheid.nl/terms/bwb/id/BWBR0005662/2043244/2002-12-31/2002-12-31',
 'http://linkeddata.overheid.nl/terms/bwb/id/BWBR0006829/1140304/1999-10-01/1999-10-01',
 'http://linkeddata.overheid.nl/terms/bwb/id/BWBR0006829/1140304/2003-01-01/2003-01-01']

In [14]:
links_titles.reset_index().columns

Index(['source', 'title', 'id'], dtype='object')

In [15]:
links_case_title = links_titles.reset_index()[['source', 'title']]
links_case_title.columns = ['source', 'target']
links_case_title.head()

,source,target
0,http://linkeddata.overheid.nl/terms/jurisprude...,Protocol bij het Verdrag tot bescherming van d...
1,http://linkeddata.overheid.nl/terms/jurisprude...,Verdrag tot bescherming van de rechten van de ...
2,http://linkeddata.overheid.nl/terms/jurisprude...,Verdrag tot bescherming van de rechten van de ...
3,http://linkeddata.overheid.nl/terms/jurisprude...,Verdrag tot bescherming van de rechten van de ...
4,http://linkeddata.overheid.nl/terms/jurisprude...,Verdrag tot bescherming van de rechten van de ...


In [16]:
nodes_articles_titles = art_nodes[['title', 'label']].drop_duplicates()
nodes_articles_titles.columns = ['id', 'label']

In [17]:
nodes_articles_titles['book'] = nodes_articles_titles.label.str.split(',').map(lambda l: l[0])

In [18]:
links_case_title.to_csv(os.path.join(filepath, 'case_to_article_title_links.csv'), index=False)
nodes_articles_titles.to_csv(os.path.join(filepath, 'article_title_nodes.csv'), index=False)

In [19]:
g = nx.from_pandas_edgelist(links_case_title, source='source', target='target')

In [20]:
case_ids = links_case_title['source'].unique()
article_ids = links_case_title['target'].unique()
nx.set_node_attributes(g, {n: 'case' for n in case_ids}, name='type')
nx.set_node_attributes(g, {n: 'article' for n in article_ids}, name='type')

In [21]:
print(nx.info(g))

Name: 
Type: Graph
Number of nodes: 457657
Number of edges: 1728187
Average degree:   7.5523


In [22]:
# Convert to iGraph
g_ig = ig.Graph.TupleList(g.edges())

att_list = set(np.array([list(d.keys()) for n, d in g.nodes(data=True)]).flatten())

for att in att_list:
    att_dict = nx.get_node_attributes(g, att)
    g_ig.vs[att] = [att_dict[n] for n in g_ig.vs['name']]

In [23]:
ccs = list(nx.connected_components(g))

In [24]:
ccs_sizes = np.array([len(c) for c in ccs])
print("Number of connected components:", len(ccs))
print("Relative size of largest component:", np.max(ccs_sizes)/np.sum(ccs_sizes))

Number of connected components: 530
Relative size of largest component: 0.996812022978


In [26]:
ccs_dict = {}
for i in range(len(ccs)):
    for n in ccs[i]:
        ccs_dict[n] = i

In [27]:
len(ccs_dict)

457657

## Community detction

In [28]:
p_01, p_0, p_1 = louvain.CPMVertexPartition.Bipartite(g_ig, resolution_parameter_01=0.01)
p_01.summary()

'Clustering with 457657 elements and 457657 clusters'

In [29]:
optimiser = louvain.Optimiser()
diff = optimiser.optimise_partition_multiplex([p_01, p_0, p_1], layer_weights=[1, -1, -1])

In [30]:
g_ig.vs['community'] = p_01.membership

In [31]:
p_01.summary()

'Clustering with 457657 elements and 6849 clusters'

In [32]:
p_0.summary()

'Clustering with 457657 elements and 6849 clusters'

In [33]:
len(p_0.membership)

457657

In [34]:
cluster_df = pd.DataFrame({'name': g_ig.vs['name'], 'community': g_ig.vs['community'],  'type': g_ig.vs['type']})

In [36]:
cluster_df['cc'] = [ccs_dict[name] for name in cluster_df['name']]

In [37]:
cluster_df.to_csv(os.path.join(filepath, 'bimodal_clusters_titles.csv'), index=False)

In [38]:
cluster_df.head()

,community,name,type,cc
0,2,http://linkeddata.overheid.nl/terms/jurisprude...,case,0
1,2,"Algemene wet bestuursrecht, Artikel 8:75",article,0
2,124,http://linkeddata.overheid.nl/terms/jurisprude...,case,0
3,124,"Burgerlijk Wetboek Boek 7, Artikel 653",article,0
4,91,http://linkeddata.overheid.nl/terms/jurisprude...,case,0


In [39]:
case_nodes_merged = case_nodes.merge(cluster_df[cluster_df['type']=='case'].drop('type', axis=1), how='left', left_on='lido_id', right_on='name', suffixes=('', '_bimodal'))
case_nodes_merged = case_nodes_merged.drop('name', axis=1).rename(columns={'community': 'community_bimodal'})
case_nodes_merged = case_nodes_merged.dropna(subset=['community_bimodal'])

In [40]:
case_nodes_merged.head()

,ecli,lido_id,country,court,year,community_bimodal,cc
0,ECLI:NL:GHSGR:2010:BM2753,http://linkeddata.overheid.nl/terms/jurisprude...,NL,GHSGR,2010.0,14.0,0.0
1,ECLI:NL:RBHAA:2007:BC0491,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBHAA,2007.0,66.0,0.0
2,ECLI:NL:RBMAA:2007:BC1654,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBMAA,2007.0,490.0,0.0
3,ECLI:NL:CRVB:2016:4389,http://linkeddata.overheid.nl/terms/jurisprude...,NL,CRVB,2016.0,2.0,0.0
4,ECLI:NL:RBLIM:2014:11172,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBLIM,2014.0,1.0,0.0


In [41]:
art_nodes_merged = nodes_articles_titles.merge(cluster_df[cluster_df['type']=='article'].drop('type', axis=1), how='left', left_on='id', right_on='name', suffixes=('', '_bimodal'))
art_nodes_merged = art_nodes_merged.drop('name', axis=1).dropna(subset=['community'])

In [42]:
art_nodes_merged.to_csv(os.path.join(filepath, 'article_nodes_nodup_min5_bimodal_titles.csv'), index=False)
case_nodes_merged.to_csv(os.path.join(filepath, 'case_nodes_simple_bimodal_titles.csv'), index=False)

In [43]:
art_nodes_merged.head()

,id,label,book,community,cc
86,Besluit afschaffing binnenlandse paspoorten en...,Besluit afschaffing binnenlandse paspoorten en...,Besluit afschaffing binnenlandse paspoorten en...,435.0,0.0
143,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,2090.0,0.0
145,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,2090.0,0.0
146,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,2090.0,0.0
148,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,Wetboek van Burgerlijke Rechtsvordering (geldt...,350.0,0.0
